In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sys
from time import sleep

In [2]:
headers = {
    "user-agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"
}

parms = {
    "pagenum" : 1,
    "Depth" : 3111
}

In [3]:
url = "http://disability.seoul.go.kr/amenity/sightSeeingConven/sightSeeingConven.jsp"

In [4]:
def getUrl(url, parameter=None, num_retries=2, select=None):
    html = requests.get(url, params=parameter, headers=headers)
    
    if 500 <= html.status_code  < 600 and num_retries > 0:
        print(html.status_code, html.reason)
        return getUrl(url, params, num_retries=1)
        
    dom = BeautifulSoup(html.text, "lxml")
    links = dom.select(select)
    
    return links

In [5]:
select_css = { 'place': 'ul.travel-list2 > li > dl > dt > a ',
               'address' : 'ul.travel-list2 > li > dl > dd > p',
               'phone' : '.tel',
               'facility': 'ul.travel-list2 > li > dl > dd > ul.facil' }

In [6]:
place_Depth_code = {
    "korean" : 3111,
    "western" : 3211,
    "chinese"  : 3311,
    "japanese" : 3411,
    "flour_based" : 3511,
    "other" : 3611,
}

# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  

In [7]:
korean = []
western = []
chinese = []
japanese = []
flour_based = []
other = []

In [10]:
# 1. 한식 2. 양식 3. 중식 4. 일식 5. 분식 6. 기타  
# type 구조 [{ }] 리스안에 딕셔너리 구조 
final_dict = {
'korean' : None ,
'western' : None ,
'chinese' : None ,
'japanese' : None ,
'flour_based' : None ,
'other' : None,

}

In [ ]:
for place in place_Depth_code:
    place_code = place_Depth_code[place]
    parms['Depth'] = place_code
    result = []  
    while True:
        for select in select_css:
            get_info = getUrl(url, parameter = parms, select = select_css[select])
            # 장소, 장소_사이트, 주소, 전화번호, 편의시설
            if 'place' is select:
                place_name = [info.text for info in get_info] # 장소
                place_href = [info['href'] for info in get_info] # 장소_사이트
            if 'address' is select:
                address_pre = [re.sub(u'\xa0 \r\n\t*',"",info.text) for info in get_info] # 주소
                address =[re.sub(u'주소 :  ',"",info) for info in address_pre]
#             if 'phone' is select:
#                 phone = [info.text for info in get_info] # 전화번호
#             if 'facility' is select:
#                 facility_pre_select = [info for info in get_info] # 편의시설
                
        if get_info: 
            for info_num in range(0, len(place_name)):
#                 facility_select =  facility_pre_select[info_num].select('li > img')
#                 facility_select = [row_facility['alt'] for row_facility in facility_select]
#                 print(facility_select)
                info_dict = {'place_name' : place_name[info_num],
                             'place_href' : 'http://disability.seoul.go.kr'+place_href[info_num],
                             'place_address' : address[info_num],
#                              'place_facility' : facility_select
                             }  #     #
                result.append(info_dict)
            print(info_dict)
            parms['pagenum'] = parms['pagenum'] + 1
            # pagenum ->for next page
        else:
            parms['pagenum'] = 1
            break
    final_dict[place] = result

In [365]:
number_len =0 
for place in place_Depth_code:
    number_len += len(final_dict[place])
print(number_len)

414


In [13]:
facility_select =  "div.imgIcoBview" # 필요한 부분만 자르기

In [14]:
facility_available_name_select = "dt > a" # 시설물 설치된 이름
facility_is_available_select = "dt > span" # 시설물 이용가능 or 이용 불편 
facility_info_select = "ul.bulListDot > li" # 시설물 상세 정보 
# place_name  = "div.viewTitle p.sisulTitle" # 시설물 상세 정보 

In [ ]:
final_dicts = {}
info_result = [] 
for place in place_Depth_code:

    for i, info in enumerate(final_dict[place]):
        info_result = [] 
        get_info = getUrl(info['place_href'], select = facility_select)
        facility_available_name= [info.text for info in get_info[0].select(facility_available_name_select)]
        facility_is_available = [info.text for info in get_info[0].select(facility_is_available_select)]
        facility_info = [info for info in get_info[0].select(facility_info_select)]
        facility_info = [info.__str__() for info in facility_info]
        facility_info = [ re.sub('^<li>|-|(<br/>)?</li>$' , '' , info).split('<br/>') for info in facility_info]
        
        for info_num in range(0, len(facility_available_name)):
            info_dict  = {'facility_available_name' : facility_available_name[info_num],
                     'facility_is_available' : facility_is_available[info_num],
                     'facility_info' : facility_info[info_num]
                     }
            info_result.append(info_dict)
        a = str(i)
        final_dict[place][i]['facility_detail_info'] = info_result
        info_result = None 
# print( final_dict[place][i]['facility_detail_info'])
#         print( final_dict[place][i]['facility_detail_info'])
#         print(final_dict[place][i]['facility_detail_info'])

In [18]:
import json
import urllib.request
import urllib.request
import datetime
import time
import json

In [19]:
def get_request_url(url):
    
    client_id="R63uNWNQiaTp1EeZl7bW"
    client_secret = "1Im1JohssC"
    
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)
    try: 
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print ("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None
    
def getGeoData(address):
    
    base = "https://openapi.naver.com/v1/map/geocode"
    node = ""
    parameters = "?query=%s" % urllib.parse.quote(address)
    url = base + node + parameters
    
    retData = get_request_url(url)
    
    if (retData == None):
        return None
    else:
        return json.loads(retData)

In [ ]:
for place in place_Depth_code:
    info_result = [] 
    print(place)
    
    for info_num,info in enumerate(final_dict[place]):
        print(info['place_address'])
        jsonResult = getGeoData(info['place_address'])
        
        try:
            for item in jsonResult['result']['items']:
                    final_dict[place][info_num]['lat'] =  str(item['point']['y'])
                    final_dict[place][info_num]['lon'] =  str(item['point']['x'])
        except Exception as e:
            print("예외!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            final_dict[place][info_num]['lat'] = "NaN"
            final_dict[place][info_num]['lon']=  "NaN"    
            print(e)

In [21]:
with open('dish_place_tests.json', 'w', encoding="utf-8") as fp:
    json.dump(final_dict, fp, ensure_ascii=False, indent="\t")

In [1]:
import json

In [2]:
with open('dish_place_tests.json', 'r', encoding='utf-8') as fp:
    final_dict = json.load(fp)

In [3]:
type(final_dict)

dict

# 1. 주요시설 이름/ 주요시설 주소/ 경도/ 위도 / 카테고리

In [4]:
import pandas as pd

In [58]:
datas = DataFrame()
for info in final_dict.keys():
    data = pd.DataFrame.from_dict(final_dict[info])
    data = pd.DataFrame(data, columns=['place_name' , 'place_address', 'lat', 'lon'])
    data['category'] = info+"_food" 
    data['lat'] = data['lat'].astype(float)
    data['lon'] = data['lon'].astype(float)
    
    datas = datas.append(data)

In [59]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 100
Data columns (total 5 columns):
place_name       414 non-null object
place_address    414 non-null object
lat              407 non-null float64
lon              407 non-null float64
category         414 non-null object
dtypes: float64(2), object(3)
memory usage: 19.4+ KB


In [60]:
# 주요시설 id_ end number =4855

In [61]:
index = [i for i in range(4856, 5270)]

In [62]:
datas['place_id'] = index

In [63]:
datas

,place_name,place_address,lat,lon,category,place_id
0,FR푸드시스템,서울특별시 강서구 공항대로 318,37.557800,126.842915,korean_food,4856
1,OK목장,서울특별시 송파구 백제고분로 547,NaN,NaN,korean_food,4857
2,The만나,서울특별시 노원구 석계로 103,37.622842,127.060726,korean_food,4858
3,㈜이리오어라(한정식),서울특별시 강남구 언주로153길 12,37.522956,127.032208,korean_food,4859
4,가야옥,서울특별시 도봉구 도당로 2,37.658405,127.035329,korean_food,4860
5,가족천하,서울특별시 영등포구 가마산로 330,37.498813,126.896346,korean_food,4861
6,갈비둥지,서울특별시 노원구 덕릉로 692,37.659410,127.075689,korean_food,4862
7,갈비사랑,서울특별시 서초구 강남대로 204,37.482488,127.036736,korean_food,4863
8,갈비세상,서울특별시 구로구 경인로 281,37.498519,126.851831,korean_food,4864
9,갈비예찬,서울특별시 도봉구 노해로 259,37.651681,127.036992,korean_food,4865


In [64]:
datas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 100
Data columns (total 6 columns):
place_name       414 non-null object
place_address    414 non-null object
lat              407 non-null float64
lon              407 non-null float64
category         414 non-null object
place_id         414 non-null int64
dtypes: float64(2), int64(1), object(3)
memory usage: 22.6+ KB


In [103]:
datas.to_csv("dish_place.csv",encoding = "utf-8")

# 2. 주요시설 이름/주요시설물 설치/ (이용가능 /이용불편)

In [76]:
data_facility = DataFrame()
for test in final_dict.keys():
    for i,category in enumerate(final_dict[test]):
        
        data_pre_facility = pd.DataFrame.from_dict(category['facility_detail_info'])
        data_pre_facility = pd.DataFrame(data_pre_facility, columns=['facility_available_name', 'facility_is_available'])
        data_pre_facility['place_name'] = category['place_name']

        data_pre_facility['place_id'] = datas.iloc[i,5]
          
        data_facility = data_facility.append(data_pre_facility)
        
data_facility['facility_available_name'] += "_"+data_facility['place_name']

In [77]:
data_facility.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 987 entries, 0 to 2
Data columns (total 4 columns):
facility_available_name    987 non-null object
facility_is_available      987 non-null object
place_name                 987 non-null object
place_id                   987 non-null int64
dtypes: int64(1), object(3)
memory usage: 38.6+ KB


In [79]:
data_facility.reset_index(inplace=True)

In [81]:
x = data_facility.apply(lambda x : str(x['place_id'])+"-"+str(x['index']),axis=1)

In [82]:
data_facility['facility_id'] =x

In [85]:
data_facility.drop(columns=['index'], inplace=True)

In [99]:
data_facility

,facility_available_name,facility_is_available,place_name,place_id,facility_id
0,장애인 전용 주차구역_FR푸드시스템,이용가능,FR푸드시스템,4856,4856-0
1,주출입구 높이차이 제거_FR푸드시스템,이용가능,FR푸드시스템,4856,4856-1
2,주출입구 접근로_OK목장,이용가능,OK목장,4857,4857-0
3,장애인 전용 주차구역_OK목장,이용가능,OK목장,4857,4857-1
4,주출입구 접근로_The만나,이용불편,The만나,4858,4858-0
5,장애인 전용 주차구역_The만나,이용가능,The만나,4858,4858-1
6,주출입구 높이차이 제거_The만나,이용불편,The만나,4858,4858-2
7,주출입구 접근로_㈜이리오어라(한정식),이용가능,㈜이리오어라(한정식),4859,4859-0
8,장애인 전용 주차구역_㈜이리오어라(한정식),이용가능,㈜이리오어라(한정식),4859,4859-1
9,주출입구 접근로_가야옥,이용불편,가야옥,4860,4860-0


In [101]:
pd.DataFrame.to_csv(data_facility, "dish_place_facility_available.csv", encoding = "utf-8")

# 3. 주요시설 상세정보 / 주요시설물 설치

In [89]:
import re

In [92]:
data_facility_detail = DataFrame()
for test in final_dict.keys():
    for category in final_dict[test]:

        for facility in category['facility_detail_info']:
            a = [info for info in facility['facility_info']]

            data_pre_facility = pd.DataFrame.from_dict(a)
           
            data_pre_facility['facility_available_name'] = facility['facility_available_name']+"_"+category['place_name']
            get_facility_id = data_facility[data_pre_facility['facility_available_name'][0] == data_facility['facility_available_name']]['facility_id']
            get_facility_id = re.findall(r'([0-9]+-[0-9]+)',str(get_facility_id))
            get_facility_id = [get_facility_id[0] for i in data_pre_facility['facility_available_name']]
            data_pre_facility['facility_id'] = get_facility_id
            data_facility_detail = data_facility_detail.append(data_pre_facility)
            

In [93]:
data_facility_detail = data_facility_detail.rename(index=str, columns={0: "facility_detail_info"})

In [94]:
data_facility_detail = data_facility_detail.reset_index(drop=True)

In [95]:
data_facility_detail

,facility_detail_info,facility_available_name,facility_id
0,총주차대수5,장애인 전용 주차구역_FR푸드시스템,4856-0
1,단차없음 또는 경사로 1/12이하 설치,주출입구 높이차이 제거_FR푸드시스템,4856-1
2,유효폭 1.2m 이상,주출입구 높이차이 제거_FR푸드시스템,4856-1
3,손잡이 설치,주출입구 높이차이 제거_FR푸드시스템,4856-1
4,평탄함,주출입구 접근로_OK목장,4857-0
5,단차없음,주출입구 접근로_OK목장,4857-0
6,유효폭 1.2m 이상,주출입구 접근로_OK목장,4857-0
7,기울기 1/18이하,주출입구 접근로_OK목장,4857-0
8,보/차도 구분,주출입구 접근로_OK목장,4857-0
9,총주차대수30,장애인 전용 주차구역_OK목장,4857-1


In [102]:
pd.DataFrame.to_csv(data_facility_detail, "dish_place_facility_detail_info.csv",encoding = "utf-8")